In [1]:
# import necessary libraries
import pandas as pd
import os
import glob
import re

In [2]:
# use glob to get all the csv files 
# in the folder
path = os.getcwd()+'/pages2'
csv_files = glob.glob(os.path.join(path, "*.csv"))

In [3]:
# import all csvs from file
df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)

In [4]:
# lowercase columns
df.columns = df.columns.str.lower()

In [5]:
# clean text
df['title']=[re.sub(r'[^\w\s]', '', str(x)) for x in df['title']]
df['header']=[re.sub(r'[^\w\s]', '', str(x)) for x in df['header']]
df['body']=[re.sub(r'[^\w\s]', '', str(x)) for x in df['body']]

# strip white space between text
df['body']=df['body'].apply(lambda x: re.sub(' +', ' ', x))
# string_to_remove_1 = 'Navigation Individuals families Business Intermediaries Healthcare professionals Find a healthcare professional or service Help support Contact us Search Bupa Search Sign in Sign in Individuals and family I have Bupa cover or Im insured through my employer Partners and Intermediaries Providers Online Im a Bupa recognised healthcare professional or provider Bupa Connect I manage my employees cover or business clients Consumer Intermediary Portal Im an intermediary with consumer clients Skip to content Accessibility Help Health Insurance Dental Care homes Pay as you go Health information Individuals families Business Intermediaries Healthcare professionals Find a healthcare professional or service '
# df['body'] = df['body'].replace(string_to_remove_1, '', regex=True)

# create a text column
# df['text']=df['title']+' '+df['header']
df['text']=df['body'].fillna('missing')

In [6]:
def word_lengths(s):
   
    # use a generator expression with map to get the lengths of all the words
    lengths = map(len, (word for word in s.split()))
    return list(lengths)

In [7]:
def hash_filtering(string):
    
    try:

        # find the length of each word in string
        word_length_list = word_lengths(string)

        # find the index of last word which is 36 letters long indicating a break
        final_index = max(index for index, item in enumerate(word_length_list) if item == 36)

        # total characters used for words and spaces
        length_of_words=sum(word_length_list[:final_index+1])
        length_of_spaces=final_index

        # filter list based on index
        output = string[length_of_words+length_of_spaces+1:]

        return output
    
    except:
        
        return string

In [8]:
def search_string_filtering(string):

    try:
        
        search_term='You are here'
        output=string[string.find(search_term)+len(search_term)+1:]

        return output
    
    except:
        return string

In [9]:
def footer_string_filtering(string):
    
    try:
        
        search_term='Health Health overview Health insurance Health cash '
        output=string[:string.find(search_term)]
        
        return output
    except:
        return string

In [10]:
df['text'] = df.apply(lambda row : hash_filtering(row['text']), axis = 1)
df['text'] = df.apply(lambda row : search_string_filtering(row['text']), axis = 1)
df['text'] = df.apply(lambda row : footer_string_filtering(row['text']), axis = 1)

string_to_remove_1 = 'Home Dental Bupa Dental Care Practices'
df['text'] = df['text'].replace(string_to_remove_1, '', regex=True)

# set a max limit of strings
df['text'] = df.text.str[:32000]

# Starts with:
# Bupa Dental Care
# Home Health Information
# txt = "Hello, welcome to my world."
# x = txt.startswith("Hello")
# x[]

In [11]:
df=df.drop_duplicates()

In [12]:
# export csv
cols=['url', 'title', 'header','text']
df[cols].to_csv(os.getcwd()+'/data/data.csv', index=False)